In [ ]:
from sklearn.naive_bayes import GaussianNB
import pandas as pd
from sklearn.metrics import (
    balanced_accuracy_score
)
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import shap
from sklearn.linear_model import LogisticRegression
import seaborn as sns
from sklearn import svm
from sklearn import neighbors
import numpy as np
import scipy.stats as stats

import warnings
warnings.filterwarnings("ignore")
import pickle
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler

from evidently.report import Report
from evidently.metrics import DataDriftTable
from evidently.metrics import DatasetDriftMetric

import matplotlib.pyplot as plt
from matplotlib import colors

from scipy import stats
from pytorch_tabnet.tab_model import TabNetClassifier
import optuna 

In [4]:
dependent_feature = ['Passed first attempt?']

In [ ]:
independent_features = ['BINGEING OF SESSIONS',
 'CONSTANCY OF CLICKS',
 'CONSTANCY OF SESSION LENGTH',
 'MEDIAN DIFF. BETWEEN ACTIVE DAYS',
 'MEDIAN NUMBER OF ACTIONS PER SESSION',
 'MEDIAN NUMBER OF ACTIVE DAYS PER WEEK',
 'MEDIAN SESSION DURATION',
 'PROPORTION OF ACTIVE DAYS',
 'PROPORTION OF FIRST-DAY-OF-WEEK ACTIVITY',
 'PROPORTION OF POSTS READ',
 'PROPORTION OF WEEKS WITH FIRST-DAY ACTIVITY',
 'REGULARITY OF SESSIONS',
 'TOTAL NUMBER OF CREATED POSTS',
 'TOTAL NUMBER OF SESSIONS',
 'TOTAL SESSIONS DURATION',
 'UNIFORMITY OF SESSIONS']

# Analysis parameter specification

In [5]:
year_start = '1920'
year_finish = '2021'
course_name = 'De globale economie'
exam_month = 'juni'

# Data

In [6]:
year = '1819'
data_1819 = pd.read_excel(f'data/{course_name}_{year}_data.xlsx')

In [21]:
year = '1920'
data_1920 = pd.read_excel(f'data/{course_name}_{year}_data.xlsx')

In [35]:
year = '2021'
data_2021 = pd.read_excel(f'data/{course_name}_{year}_data.xlsx')

In [49]:
if (year_start == '1920' and year_finish == '2021'):

    X_y1 = data_1920[independent_features].copy().to_numpy()
    y_y1 = data_1920[dependent_feature].copy().to_numpy()

    X_y2 = data_2021[independent_features].copy().to_numpy()
    y_y2 = data_2021[dependent_feature].copy().to_numpy()

elif (year_start == '1819' and year_finish == '1920'):
    X_y1 = data_1819[independent_features].copy().to_numpy()
    y_y1 = data_1819[dependent_feature].copy().to_numpy()

    X_y2 = data_1920[independent_features].copy().to_numpy()
    y_y2 = data_1920[dependent_feature].copy().to_numpy()

# Data drift analysis

## 1819 vs. 1920

In [50]:
data_drift_dataset_report = Report(metrics = [
    DatasetDriftMetric(stattest_threshold = 0.025),
    DataDriftTable()
])

In [51]:
data_drift_dataset_report.run(reference_data = data_1819[independent_features],
                              current_data = data_1920[independent_features])

In [ ]:
# the report can be seen in online appendices
data_drift_dataset_report.show(mode = 'inline')

## 1920 vs. 2021

In [53]:
data_drift_dataset_report.run(reference_data = data_1920[independent_features],
                              current_data = data_2021[independent_features])

In [ ]:
# the report can be seen in online appendices
data_drift_dataset_report.show(mode = 'inline')

# Modeling

In [50]:
def psi(score_initial, score_new, num_bins = 10):
    
    eps = 1e-4
    
    score_initial.sort()
    score_new.sort()
    
    min_val = min(min(score_initial), min(score_new))
    max_val = max(max(score_initial), max(score_new))

    bins = [min_val + (max_val - min_val)*(i)/num_bins for i in range(num_bins+1)]

    bins[0] = min_val - eps 
    bins[-1] = max_val + eps 
          
    bins_initial = pd.cut(score_initial, bins = bins, labels = range(1,num_bins+1))
    df_initial = pd.DataFrame({'initial': score_initial, 'bin': bins_initial})
    grp_initial = df_initial.groupby('bin').count()
    grp_initial['percent_initial'] = grp_initial['initial'] / sum(grp_initial['initial'])
    
    bins_new = pd.cut(score_new, bins = bins, labels = range(1,num_bins+1))
    df_new = pd.DataFrame({'new': score_new, 'bin': bins_new})
    grp_new = df_new.groupby('bin').count()
    grp_new['percent_new'] = grp_new['new'] / sum(grp_new['new'])
    
    psi_df = grp_initial.join(grp_new, on = "bin", how = "inner")
    
    psi_df['percent_initial'] = psi_df['percent_initial'].apply(lambda x: eps if x == 0 else x)
    psi_df['percent_new'] = psi_df['percent_new'].apply(lambda x: eps if x == 0 else x)
    
    psi_df['psi'] = (psi_df['percent_initial'] - psi_df['percent_new']) * np.log(psi_df['percent_initial'] / psi_df['percent_new'])
    
    return np.sum(psi_df['psi'].values)

In [51]:
def run_models_scaled(model, model_name, param_grid):
    inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
    outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)

    psi_scores_y1_y2 = []
    psi_scores_y1_y2_retrained = []
    psi_scores_y1_y2_cv = []

    outer_scores_y1 = []
    shap_scores_y1 = []

    outer_scores_y2 = []
    shap_scores_y2 = []

    outer_scores_y2_retrained = []
    shap_scores_y2_retrained = []

    outer_scores_y2_cv = []
    shap_scores_y2_cv = []

    ix_training_y1, ix_test_y1 = [], []
    ix_training_y2, ix_test_y2 = [], []

    shaps_y1_per_fold = {}
    shaps_y2_per_fold = {}
    shaps_y2_retrained_per_fold = {}
    shaps_y2_cv_per_fold = {}

    for fold in outer_cv.split(X_y1, y_y1):
        ix_training_y1.append(fold[0]), ix_test_y1.append(fold[1])

    for fold in outer_cv.split(X_y2, y_y2):
        ix_training_y2.append(fold[0]), ix_test_y2.append(fold[1])

    
    for i in range(10):  

        train_idX_y1, test_idX_y1 = ix_training_y1[i], ix_test_y1[i]
        train_idX_y2, test_idX_y2 = ix_training_y2[i], ix_test_y2[i]

        mms = MinMaxScaler()

        X_train_y1_scaled = mms.fit_transform(X_y1[train_idX_y1].copy())
        X_train_y1_scaled = pd.DataFrame(X_train_y1_scaled)
        X_train_y1_scaled.columns = independent_features

        X_test_y1_scaled = mms.transform(X_y1[test_idX_y1].copy())
        X_test_y1_scaled = pd.DataFrame(X_test_y1_scaled)
        X_test_y1_scaled.columns = independent_features

        mms = MinMaxScaler()

        X_train_y2_scaled = mms.fit_transform(X_y2[train_idX_y2].copy())
        X_train_y2_scaled = pd.DataFrame(X_train_y2_scaled)
        X_train_y2_scaled.columns = independent_features

        X_test_y2_scaled = mms.transform(X_y2[test_idX_y2].copy())
        X_test_y2_scaled = pd.DataFrame(X_test_y2_scaled)
        X_test_y2_scaled.columns = independent_features

        # step 1)
        param_grid = param_grid
        gcv = GridSearchCV(estimator=model,
                            param_grid=param_grid,
                            scoring='balanced_accuracy',
                            n_jobs=-1,
                            cv=inner_cv,
                            verbose=0,
                            refit=True)
        gcv.fit(X_train_y1_scaled, y_y1[train_idX_y1])
        best_model = gcv.best_estimator_
        print(f'Step {i} for model {model_name}')
        print('\n        Best ACCURACY y1 model %.2f%%' % (gcv.best_score_ * 100))
        print('        Best parameters 1819 model:', gcv.best_params_)
        
        # step 2) 
        y_pred = best_model.predict(X_test_y1_scaled)
        outer_scores_y1.append(balanced_accuracy_score(y_y1[test_idX_y1], y_pred))
        print('ACCURACY y1 (on outer test fold) %.2f%%' % (outer_scores_y1[-1]*100))

        # step 3) 
        explainer_y1 = shap.KernelExplainer(best_model.predict_proba, shap.sample(X_train_y1_scaled,100))
        shap_values_y1 = explainer_y1.shap_values(X_test_y1_scaled)
        for SHAPs in shap_values_y1[1]:
            shap_scores_y1.append(SHAPs)
        shaps_y1_per_fold[i] = shap_values_y1[1]

        # step 4)  
        y_pred = best_model.predict(X_test_y2_scaled)
        outer_scores_y2.append(balanced_accuracy_score(y_y2[test_idX_y2], y_pred))
        print('ACCURACY y2 (on outer test fold) %.2f%%' % (outer_scores_y2[-1]*100))
        probas_y1 = best_model.predict_proba(X_test_y1_scaled)[:,1]
        probas_y2 = best_model.predict_proba(X_test_y2_scaled)[:,1]
        psi_scores_y1_y2.append(psi(probas_y1,
                                     probas_y2))

        # step 5)
        shap_values_y2 = explainer_y1.shap_values(X_test_y2_scaled)
        for SHAPs in shap_values_y2[1]:
            shap_scores_y2.append(SHAPs) 
        shaps_y2_per_fold[i] = shap_values_y2[1]

        # step 6)
        best_model.fit(X_train_y2_scaled, y_y2[train_idX_y2])

        # step 7)
        y_pred = best_model.predict(X_test_y2_scaled)
        outer_scores_y2_retrained.append(balanced_accuracy_score(y_y2[test_idX_y2], y_pred))
        print('ACCURACY y2 retrained (on outer test fold) %.2f%%' % (outer_scores_y2_retrained[-1]*100))
        probas_y2_retrained = best_model.predict_proba(X_test_y2_scaled)[:,1]
        psi_scores_y1_y2_retrained.append(psi(probas_y1,
                                               probas_y2_retrained))

        # step 8)
        explainer_y2_retrained = shap.KernelExplainer(best_model.predict_proba, shap.sample(X_train_y2_scaled,100))
        shap_values_y2_retrained = explainer_y2_retrained.shap_values(X_test_y2_scaled)
        for SHAPs in shap_values_y2_retrained[1]:
            shap_scores_y2_retrained.append(SHAPs) 
        shaps_y2_retrained_per_fold[i] = shap_values_y2_retrained[1]

        #step 9) 

        gcv_y2 = GridSearchCV(estimator=model,
                        param_grid=param_grid,
                        scoring='balanced_accuracy',
                        n_jobs=-1,
                        cv=inner_cv,
                        verbose=0,
                        refit=True)
        
        gcv_y2.fit(X_train_y2_scaled, y_y2[train_idX_y2]) 
        best_model_y2 = gcv_y2.best_estimator_
        print('\n        Best ACCURACY y2 CV model %.2f%%' % (gcv_y2.best_score_ * 100))
        print('        Best parameters:', gcv_y2.best_params_)

        # step 10)
        y_pred = best_model_y2.predict(X_test_y2_scaled)
        outer_scores_y2_cv.append(balanced_accuracy_score(y_y2[test_idX_y2], y_pred))
        print('ACCURACY y2 after CV (on outer test fold) %.2f%%' % (outer_scores_y2_cv[-1]*100))
        probas_y2_cv = best_model_y2.predict_proba(X_test_y2_scaled)[:,1]
        psi_scores_y1_y2_cv.append(psi(probas_y1,
                                        probas_y2_cv))

        #step 11)
        explainer_y2_cv = shap.KernelExplainer(best_model_y2.predict_proba, shap.sample(X_train_y2_scaled,100))
        shap_values_y2_cv = explainer_y2_cv.shap_values(X_test_y2_scaled)
        for SHAPs in shap_values_y2_cv[1]:
            shap_scores_y2_cv.append(SHAPs)
        shaps_y2_cv_per_fold[i] = shap_values_y2_cv[1]
    
    return outer_scores_y1, shap_scores_y1, outer_scores_y2,\
    shap_scores_y2, outer_scores_y2_retrained, shap_scores_y2_retrained,\
    outer_scores_y2_cv, shap_scores_y2_cv, ix_training_y1, ix_test_y1,\
    ix_training_y2, ix_test_y2, shaps_y1_per_fold, shaps_y2_per_fold,\
          shaps_y2_retrained_per_fold, shaps_y2_cv_per_fold,\
          psi_scores_y1_y2,\
              psi_scores_y1_y2_retrained, psi_scores_y1_y2_cv

In [52]:
def run_models(model, model_name, param_grid):
    inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
    outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)

    psi_scores_y1_y2 = []
    psi_scores_y1_y2_retrained = []
    psi_scores_y1_y2_cv = []

    outer_scores_y1 = []
    shap_scores_y1 = []

    outer_scores_y2 = []
    shap_scores_y2 = []

    outer_scores_y2_retrained = []
    shap_scores_y2_retrained = []

    outer_scores_y2_cv = []
    shap_scores_y2_cv = []

    ix_training_y1, ix_test_y1 = [], []
    ix_training_y2, ix_test_y2 = [], []

    shaps_y1_per_fold = {}
    shaps_y2_per_fold = {}
    shaps_y2_retrained_per_fold = {}
    shaps_y2_cv_per_fold = {}

    for fold in outer_cv.split(X_y1, y_y1):
        ix_training_y1.append(fold[0]), ix_test_y1.append(fold[1])

    for fold in outer_cv.split(X_y2, y_y2):
        ix_training_y2.append(fold[0]), ix_test_y2.append(fold[1])

    
    for i in range(10):  

        train_idX_y1, test_idX_y1 = ix_training_y1[i], ix_test_y1[i]
        train_idX_y2, test_idX_y2 = ix_training_y2[i], ix_test_y2[i]
        
        # step 1)
        param_grid = param_grid
        gcv = GridSearchCV(estimator=model,
                            param_grid=param_grid,
                            scoring='balanced_accuracy',
                            n_jobs=-1,
                            cv=inner_cv,
                            verbose=0,
                            refit=True)
        
        gcv.fit(X_y1[train_idX_y1], y_y1[train_idX_y1]) 
        best_model = gcv.best_estimator_
        print(f'Step {i} for model {model_name}')
        print('\n        Best ACCURACY y1 model %.2f%%' % (gcv.best_score_ * 100))
        print('        Best parameters 1819 model:', gcv.best_params_)
        
        # step 2) 
        y_pred = best_model.predict(X_y1[test_idX_y1])
        outer_scores_y1.append(balanced_accuracy_score(y_y1[test_idX_y1], y_pred))
        print('ACCURACY y1 (on outer test fold) %.2f%%' % (outer_scores_y1[-1]*100))

        # step 3) 
        explainer_y1 = shap.KernelExplainer(best_model.predict_proba, shap.sample(X_y1[train_idX_y1],100))
        shap_values_y1 = explainer_y1.shap_values(X_y1[test_idX_y1])
        for SHAPs in shap_values_y1[1]:
            shap_scores_y1.append(SHAPs)
        shaps_y1_per_fold[i] = shap_values_y1[1]

        # step 4)  
        y_pred = best_model.predict(X_y2[test_idX_y2])
        outer_scores_y2.append(balanced_accuracy_score(y_y2[test_idX_y2], y_pred))
        print('ACCURACY y2 (on outer test fold) %.2f%%' % (outer_scores_y2[-1]*100))
        probas_y1 = best_model.predict_proba(X_y1[test_idX_y1])[:,1]
        probas_y2 = best_model.predict_proba(X_y2[test_idX_y2])[:,1]
        psi_scores_y1_y2.append(psi(probas_y1,
                                     probas_y2))

        # step 5)
        shap_values_y2 = explainer_y1.shap_values(X_y2[test_idX_y2])
        for SHAPs in shap_values_y2[1]:
            shap_scores_y2.append(SHAPs) 
        shaps_y2_per_fold[i] = shap_values_y2[1]

        # step 6)
        best_model.fit(X_y2[train_idX_y2], y_y2[train_idX_y2])

        # step 7)
        y_pred = best_model.predict(X_y2[test_idX_y2])
        outer_scores_y2_retrained.append(balanced_accuracy_score(y_y2[test_idX_y2], y_pred))
        print('ACCURACY y2 retrained (on outer test fold) %.2f%%' % (outer_scores_y2_retrained[-1]*100))
        probas_y2_retrained = best_model.predict_proba(X_y2[test_idX_y2])[:,1]
        psi_scores_y1_y2_retrained.append(psi(probas_y1,
                                               probas_y2_retrained))

        # step 8)
        explainer_y2_retrained = shap.KernelExplainer(best_model.predict_proba, shap.sample(X_y2[train_idX_y2],100))
        shap_values_y2_retrained = explainer_y2_retrained.shap_values(X_y2[test_idX_y2])
        for SHAPs in shap_values_y2_retrained[1]:
            shap_scores_y2_retrained.append(SHAPs) 
        shaps_y2_retrained_per_fold[i] = shap_values_y2_retrained[1]

        #step 9) 

        gcv_y2 = GridSearchCV(estimator=model,
                        param_grid=param_grid,
                        scoring='balanced_accuracy',
                        n_jobs=-1,
                        cv=inner_cv,
                        verbose=0,
                        refit=True)
        
        gcv_y2.fit(X_y2[train_idX_y2], y_y2[train_idX_y2])
        best_model_y2 = gcv_y2.best_estimator_
        print('\n        Best ACCURACY y2 CV model %.2f%%' % (gcv_y2.best_score_ * 100))
        print('        Best parameters:', gcv_y2.best_params_)

        # step 10)
        y_pred = best_model_y2.predict(X_y2[test_idX_y2])
        outer_scores_y2_cv.append(balanced_accuracy_score(y_y2[test_idX_y2], y_pred))
        print('ACCURACY y2 after CV (on outer test fold) %.2f%%' % (outer_scores_y2_cv[-1]*100))
        probas_y2_cv = best_model_y2.predict_proba(X_y2[test_idX_y2])[:,1]
        psi_scores_y1_y2_cv.append(psi(probas_y1,
                                        probas_y2_cv))

        #step 11)
        explainer_y2_cv = shap.KernelExplainer(best_model_y2.predict_proba, shap.sample(X_y2[train_idX_y2],100))
        shap_values_y2_cv = explainer_y2_cv.shap_values(X_y2[test_idX_y2])
        for SHAPs in shap_values_y2_cv[1]:
            shap_scores_y2_cv.append(SHAPs)
        shaps_y2_cv_per_fold[i] = shap_values_y2_cv[1]

    return outer_scores_y1, shap_scores_y1, outer_scores_y2,\
          shap_scores_y2, outer_scores_y2_retrained, shap_scores_y2_retrained,\
              outer_scores_y2_cv, shap_scores_y2_cv, ix_training_y1, ix_test_y1,\
                  ix_training_y2, ix_test_y2,shaps_y1_per_fold, shaps_y2_per_fold,\
          shaps_y2_retrained_per_fold, shaps_y2_cv_per_fold,\
          psi_scores_y1_y2,\
              psi_scores_y1_y2_retrained, psi_scores_y1_y2_cv

### NB

In [ ]:
nb_outer_scores_y1, nb_shap_scores_y1, nb_outer_scores_y2, nb_shap_scores_y2,\
      nb_outer_scores_y2_retrained, nb_shap_scores_y2_retrained, nb_outer_scores_y2_cv,\
          nb_shap_scores_y2_cv, nb_ix_training_y1, nb_ix_test_y1, nb_ix_training_y2,\
              nb_ix_test_y2, nb_shaps_y1_per_fold, nb_shaps_y2_per_fold,\
          nb_shaps_y2_retrained_per_fold, nb_shaps_y2_cv_per_fold,\
            nb_psi_scores_y1_y2,\
              nb_psi_scores_y1_y2_retrained, nb_psi_scores_y1_y2_cv= run_models(GaussianNB(), 'NB',
                                                                                    [{'var_smoothing': np.logspace(0,-9, num=100)}])

In [ ]:
feature_names = independent_features

resultX = pd.DataFrame(nb_shap_scores_y1, columns = feature_names)
vals = np.abs(resultX.values).mean(0)
shap_importance_nb = pd.DataFrame(list(zip(feature_names, vals)),
                                  columns=['col_name','feature_importance_vals'])
shap_importance_nb.sort_values(by=['feature_importance_vals'],
                               ascending=False, inplace=True)
shap_importance_nb['Rank'] = len(stats.rankdata(shap_importance_nb['feature_importance_vals'])) \
    - stats.rankdata(shap_importance_nb['feature_importance_vals']) + 1

resultX = pd.DataFrame(nb_shap_scores_y2, columns = feature_names)
vals = np.abs(resultX.values).mean(0)
shap_importance_nb_y2 = pd.DataFrame(list(zip(feature_names, vals)),
                                  columns=['col_name','feature_importance_vals'])
shap_importance_nb_y2.sort_values(by=['feature_importance_vals'],
                               ascending=False, inplace=True)
shap_importance_nb_y2['Rank'] = len(stats.rankdata(shap_importance_nb_y2['feature_importance_vals'])) \
    - stats.rankdata(shap_importance_nb_y2['feature_importance_vals']) + 1

resultX = pd.DataFrame(nb_shap_scores_y2_retrained, columns = feature_names)
vals = np.abs(resultX.values).mean(0)
shap_importance_retrained_nb_y2 = pd.DataFrame(list(zip(feature_names, vals)),
                                  columns=['col_name','feature_importance_vals'])
shap_importance_retrained_nb_y2.sort_values(by=['feature_importance_vals'],
                               ascending=False, inplace=True)
shap_importance_retrained_nb_y2['Rank'] = len(stats.rankdata(shap_importance_retrained_nb_y2['feature_importance_vals'])) \
    - stats.rankdata(shap_importance_retrained_nb_y2['feature_importance_vals']) + 1

resultX = pd.DataFrame(nb_shap_scores_y2_cv, columns = feature_names)
vals = np.abs(resultX.values).mean(0)
shap_importance_nb_cv_y2 = pd.DataFrame(list(zip(feature_names, vals)),
                                  columns=['col_name','feature_importance_vals'])
shap_importance_nb_cv_y2.sort_values(by=['feature_importance_vals'],
                               ascending=False, inplace=True)
shap_importance_nb_cv_y2['Rank'] = len(stats.rankdata(shap_importance_nb_cv_y2['feature_importance_vals'])) \
    - stats.rankdata(shap_importance_nb_cv_y2['feature_importance_vals']) + 1

In [ ]:
pd.DataFrame(nb_shap_scores_y1, columns=independent_features).to_excel(f'results/{course_name}_SHAP_scores_NB_{year_start}.xlsx')
pd.DataFrame(nb_shap_scores_y2, columns=independent_features).to_excel(f'results/{course_name}_SHAP_scores_NB_{year_finish}.xlsx')
pd.DataFrame(nb_shap_scores_y2_retrained, columns=independent_features).to_excel(f'results/{course_name}_SHAP_scores_NB_{year_finish}_retrained.xlsx')
pd.DataFrame(nb_shap_scores_y2_cv, columns=independent_features).to_excel(f'results/{course_name}_SHAP_scores_NB_{year_finish}_cv.xlsx')
pd.DataFrame({f'NB {year_start}': nb_outer_scores_y1, f'NB {year_finish}': nb_outer_scores_y2, f'NB {year_finish} retrained': nb_outer_scores_y2_retrained,
               f'NB {year_finish} cv': nb_outer_scores_y2_cv }).to_excel(f'results/{course_name}_NB_performance.xlsx')

In [ ]:
with pd.ExcelWriter(f'results/{course_name}_SHAP_scores_NB_{year_start}_per_fold.xlsx', engine='xlsxwriter') as writer:
    for i in range(10):

        sheet_name = f'Sheet{i+1}' 
        pd.DataFrame(nb_shaps_y1_per_fold[i]).to_excel(writer, sheet_name=sheet_name, index=False)

with pd.ExcelWriter(f'results/{course_name}_SHAP_scores_NB_{year_finish}_per_fold.xlsx', engine='xlsxwriter') as writer:
    for i in range(10):

        sheet_name = f'Sheet{i+1}'  
        pd.DataFrame(nb_shaps_y2_per_fold[i]).to_excel(writer, sheet_name=sheet_name, index=False)

with pd.ExcelWriter(f'results/{course_name}_SHAP_scores_NB_{year_finish}_retrained_per_fold.xlsx', engine='xlsxwriter') as writer:
    for i in range(10):

        sheet_name = f'Sheet{i+1}'  
        pd.DataFrame(nb_shaps_y2_retrained_per_fold[i]).to_excel(writer, sheet_name=sheet_name, index=False)

with pd.ExcelWriter(f'results/{course_name}_SHAP_scores_NB_{year_finish}_cv_per_fold.xlsx', engine='xlsxwriter') as writer:
    for i in range(10):

        sheet_name = f'Sheet{i+1}' 
        pd.DataFrame(nb_shaps_y2_cv_per_fold[i]).to_excel(writer, sheet_name=sheet_name, index=False)

In [ ]:
nb_psi = pd.DataFrame({'Model X on data X vs. Model X on data X+1':nb_psi_scores_y1_y2, 
                        'Model X on data X vs. Model X retrained on data X+1':nb_psi_scores_y1_y2_retrained,
                          'Model X on data X vs. Model X+1 on data X+1':nb_psi_scores_y1_y2_cv})
nb_psi.to_excel(f'results/{course_name} {year_start} vs. {year_finish} NB PSI.xlsx')

In [ ]:
with open(f'results/{course_name} {year_start} NB train indices.pickle', 'wb') as handle:
    pickle.dump(nb_ix_training_y1, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(f'results/{course_name} {year_start} NB test indices.pickle', 'wb') as handle:
    pickle.dump(nb_ix_test_y1, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(f'results/{course_name} {year_finish} NB train indices.pickle', 'wb') as handle:
    pickle.dump(nb_ix_training_y2, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(f'results/{course_name} {year_finish} NB test indices.pickle', 'wb') as handle:
    pickle.dump(nb_ix_test_y2, handle, protocol=pickle.HIGHEST_PROTOCOL)    

### RF

In [ ]:
rf_outer_scores_y1, rf_shap_scores_y1, rf_outer_scores_y2, rf_shap_scores_y2, rf_outer_scores_y2_retrained, \
      rf_shap_scores_y2_retrained, rf_outer_scores_y2_cv, rf_shap_scores_y2_cv, rf_ix_training_y1, rf_ix_test_y1, \
          rf_ix_training_y2, rf_ix_test_y2, rf_shaps_y1_per_fold, rf_shaps_y2_per_fold,\
          rf_shaps_y2_retrained_per_fold, rf_shaps_y2_cv_per_fold,\
            rf_psi_scores_y1_y2,\
              rf_psi_scores_y1_y2_retrained, rf_psi_scores_y1_y2_cv = run_models(RandomForestClassifier(random_state = 0), 'RF',
                                                                                    [{'max_depth': [3, 5, 10, None], 
                                                                                      'min_samples_split': [2, 5, 10],
                                                                                        'criterion': ['gini', 'entropy', 'log_loss'],
                                                                                    'max_features': ['sqrt','log2', None],
                                                                                        'n_estimators': [50, 100, 200], 
                                                                                        'max_samples': [int(X_y1.shape[0]/2) ,X_y1.shape[0]]}])

In [57]:
feature_names = independent_features

resultX = pd.DataFrame(rf_shap_scores_y1, columns = feature_names)
vals = np.abs(resultX.values).mean(0)
shap_importance_rf = pd.DataFrame(list(zip(feature_names, vals)),
                                  columns=['col_name','feature_importance_vals'])
shap_importance_rf.sort_values(by=['feature_importance_vals'],
                               ascending=False, inplace=True)
shap_importance_rf['Rank'] = len(stats.rankdata(shap_importance_rf['feature_importance_vals'])) \
    - stats.rankdata(shap_importance_rf['feature_importance_vals']) + 1

resultX = pd.DataFrame(rf_shap_scores_y2, columns = feature_names)
vals = np.abs(resultX.values).mean(0)
shap_importance_rf_y2 = pd.DataFrame(list(zip(feature_names, vals)),
                                  columns=['col_name','feature_importance_vals'])
shap_importance_rf_y2.sort_values(by=['feature_importance_vals'],
                               ascending=False, inplace=True)
shap_importance_rf_y2['Rank'] = len(stats.rankdata(shap_importance_rf_y2['feature_importance_vals'])) \
      - stats.rankdata(shap_importance_rf_y2['feature_importance_vals']) + 1

resultX = pd.DataFrame(rf_shap_scores_y2_retrained, columns = feature_names)
vals = np.abs(resultX.values).mean(0)
shap_importance_retrained_rf_y2 = pd.DataFrame(list(zip(feature_names, vals)),
                                  columns=['col_name','feature_importance_vals'])
shap_importance_retrained_rf_y2.sort_values(by=['feature_importance_vals'],
                               ascending=False, inplace=True)
shap_importance_retrained_rf_y2['Rank'] = len(stats.rankdata(shap_importance_retrained_rf_y2['feature_importance_vals'])) \
      - stats.rankdata(shap_importance_retrained_rf_y2['feature_importance_vals']) + 1

resultX = pd.DataFrame(rf_shap_scores_y2_cv, columns = feature_names)
vals = np.abs(resultX.values).mean(0)
shap_importance_rf_cv_y2 = pd.DataFrame(list(zip(feature_names, vals)),
                                  columns=['col_name','feature_importance_vals'])
shap_importance_rf_cv_y2.sort_values(by=['feature_importance_vals'],
                               ascending=False, inplace=True)
shap_importance_rf_cv_y2['Rank'] = len(stats.rankdata(shap_importance_rf_cv_y2['feature_importance_vals'])) \
      - stats.rankdata(shap_importance_rf_cv_y2['feature_importance_vals']) + 1

In [58]:
pd.DataFrame(rf_shap_scores_y1, columns=independent_features).to_excel(f'results/{course_name}_SHAP_scores_RF_{year_start}.xlsx')
pd.DataFrame(rf_shap_scores_y2, columns=independent_features).to_excel(f'results/{course_name}_SHAP_scores_RF_{year_finish}.xlsx')
pd.DataFrame(rf_shap_scores_y2_retrained, columns=independent_features).to_excel(f'results/{course_name}_SHAP_scores_RF_{year_finish}_retrained.xlsx')
pd.DataFrame(rf_shap_scores_y2_cv, columns=independent_features).to_excel(f'results/{course_name}_SHAP_scores_RF_{year_finish}_cvd.xlsx')
pd.DataFrame({f'RF {year_start}': rf_outer_scores_y1, f'RF {year_finish}': rf_outer_scores_y2, f'RF {year_finish} retrained': rf_outer_scores_y2_retrained,
               f'RF {year_finish} cv': rf_outer_scores_y2_cv }).to_excel(f'results/{course_name}_RF_performance.xlsx')

In [59]:
with pd.ExcelWriter(f'results/{course_name}_SHAP_scores_RF_{year_start}_per_fold.xlsx', engine='xlsxwriter') as writer:
    for i in range(10):

        sheet_name = f'Sheet{i+1}' 
        pd.DataFrame(rf_shaps_y1_per_fold[i]).to_excel(writer, sheet_name=sheet_name, index=False)

with pd.ExcelWriter(f'results/{course_name}_SHAP_scores_RF_{year_finish}_per_fold.xlsx', engine='xlsxwriter') as writer:
    for i in range(10):

        sheet_name = f'Sheet{i+1}'  
        pd.DataFrame(rf_shaps_y2_per_fold[i]).to_excel(writer, sheet_name=sheet_name, index=False)

with pd.ExcelWriter(f'results/{course_name}_SHAP_scores_RF_{year_finish}_retrained_per_fold.xlsx', engine='xlsxwriter') as writer:
    for i in range(10):

        sheet_name = f'Sheet{i+1}'  
        pd.DataFrame(rf_shaps_y2_retrained_per_fold[i]).to_excel(writer, sheet_name=sheet_name, index=False)

with pd.ExcelWriter(f'results/{course_name}_SHAP_scores_RF_{year_finish}_cv_per_fold.xlsx', engine='xlsxwriter') as writer:
    for i in range(10):

        sheet_name = f'Sheet{i+1}' 
        pd.DataFrame(rf_shaps_y2_cv_per_fold[i]).to_excel(writer, sheet_name=sheet_name, index=False)

In [60]:
rf_psi = pd.DataFrame({'Model X on data X vs. Model X on data X+1':rf_psi_scores_y1_y2, 
                        'Model X on data X vs. Model X retrained on data X+1':rf_psi_scores_y1_y2_retrained,
                          'Model X on data X vs. Model X+1 on data X+1':rf_psi_scores_y1_y2_cv})
rf_psi.to_excel(f'results/{course_name} {year_start} vs. {year_finish} RF PSI.xlsx')

In [61]:
with open(f'results/{course_name} {year_start} RF train indices.pickle', 'wb') as handle:
    pickle.dump(rf_ix_training_y1, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(f'results/{course_name} {year_start} RF test indices.pickle', 'wb') as handle:
    pickle.dump(rf_ix_test_y1, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(f'results/{course_name} {year_finish} RF train indices.pickle', 'wb') as handle:
    pickle.dump(rf_ix_training_y2, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(f'results/{course_name} {year_finish} RF test indices.pickle', 'wb') as handle:
    pickle.dump(rf_ix_test_y2, handle, protocol=pickle.HIGHEST_PROTOCOL)    

### LR

In [ ]:
lr_outer_scores_y1, lr_shap_scores_y1, lr_outer_scores_y2, lr_shap_scores_y2, lr_outer_scores_y2_retrained, \
    lr_shap_scores_y2_retrained, lr_outer_scores_y2_cv, lr_shap_scores_y2_cv, lr_ix_training_y1, lr_ix_test_y1, \
        lr_ix_training_y2, lr_ix_test_y2, lr_shaps_y1_per_fold, lr_shaps_y2_per_fold,\
          lr_shaps_y2_retrained_per_fold, lr_shaps_y2_cv_per_fold,\
            lr_psi_scores_y1_y2,\
              lr_psi_scores_y1_y2_retrained, lr_psi_scores_y1_y2_cv = run_models(LogisticRegression(random_state = 0), 'LR', [{ 'penalty' : ['l1','l2'], 
                                                                                    'C'       : np.logspace(-3,3,7),
                                                                                    'solver'  : ['newton-cg', 'lbfgs', 'liblinear']}])

In [63]:
feature_names = independent_features

resultX = pd.DataFrame(lr_shap_scores_y1, columns = feature_names)
vals = np.abs(resultX.values).mean(0)
shap_importance_lr = pd.DataFrame(list(zip(feature_names, vals)),
                                  columns=['col_name','feature_importance_vals'])
shap_importance_lr.sort_values(by=['feature_importance_vals'],
                               ascending=False, inplace=True)
shap_importance_lr['Rank'] = len(stats.rankdata(shap_importance_lr['feature_importance_vals'])) \
    - stats.rankdata(shap_importance_lr['feature_importance_vals']) + 1

resultX = pd.DataFrame(lr_shap_scores_y2, columns = feature_names)
vals = np.abs(resultX.values).mean(0)
shap_importance_lr_y2 = pd.DataFrame(list(zip(feature_names, vals)),
                                  columns=['col_name','feature_importance_vals'])
shap_importance_lr_y2.sort_values(by=['feature_importance_vals'],
                               ascending=False, inplace=True)
shap_importance_lr_y2['Rank'] = len(stats.rankdata(shap_importance_lr_y2['feature_importance_vals'])) \
      - stats.rankdata(shap_importance_lr_y2['feature_importance_vals']) + 1

resultX = pd.DataFrame(lr_shap_scores_y2_retrained, columns = feature_names)
vals = np.abs(resultX.values).mean(0)
shap_importance_retrained_lr_y2 = pd.DataFrame(list(zip(feature_names, vals)),
                                  columns=['col_name','feature_importance_vals'])
shap_importance_retrained_lr_y2.sort_values(by=['feature_importance_vals'],
                               ascending=False, inplace=True)
shap_importance_retrained_lr_y2['Rank'] = len(stats.rankdata(shap_importance_retrained_lr_y2['feature_importance_vals'])) \
    - stats.rankdata(shap_importance_retrained_lr_y2['feature_importance_vals']) + 1

resultX = pd.DataFrame(lr_shap_scores_y2_cv, columns = feature_names)
vals = np.abs(resultX.values).mean(0)
shap_importance_lr_cv_y2 = pd.DataFrame(list(zip(feature_names, vals)),
                                  columns=['col_name','feature_importance_vals'])
shap_importance_lr_cv_y2.sort_values(by=['feature_importance_vals'],
                               ascending=False, inplace=True)                   
shap_importance_lr_cv_y2['Rank'] = len(stats.rankdata(shap_importance_lr_cv_y2['feature_importance_vals'])) \
    - stats.rankdata(shap_importance_lr_cv_y2['feature_importance_vals']) + 1         

In [64]:
pd.DataFrame(lr_shap_scores_y1, columns=independent_features).to_excel(f'results/{course_name}_SHAP_scores_LR_{year_start}.xlsx')
pd.DataFrame(lr_shap_scores_y2, columns=independent_features).to_excel(f'results/{course_name}_SHAP_scores_LR_{year_finish}.xlsx')
pd.DataFrame(lr_shap_scores_y2_retrained, columns=independent_features).to_excel(f'results/{course_name}_SHAP_scores_LR_{year_finish}_retrained.xlsx')
pd.DataFrame(lr_shap_scores_y2_cv, columns=independent_features).to_excel(f'results/{course_name}_SHAP_scores_LR_{year_finish}_cvd.xlsx')
pd.DataFrame({f'LR {year_start}': lr_outer_scores_y1, f'LR {year_finish}': lr_outer_scores_y2, f'LR {year_finish} retrained': lr_outer_scores_y2_retrained,
               f'LR {year_finish} cv': lr_outer_scores_y2_cv }).to_excel(f'results/{course_name}_LR_performance.xlsx')

In [65]:
with pd.ExcelWriter(f'results/{course_name}_SHAP_scores_LR_{year_start}_per_fold.xlsx', engine='xlsxwriter') as writer:
    for i in range(10):

        sheet_name = f'Sheet{i+1}' 
        pd.DataFrame(lr_shaps_y1_per_fold[i]).to_excel(writer, sheet_name=sheet_name, index=False)

with pd.ExcelWriter(f'results/{course_name}_SHAP_scores_LR_{year_finish}_per_fold.xlsx', engine='xlsxwriter') as writer:
    for i in range(10):

        sheet_name = f'Sheet{i+1}'  
        pd.DataFrame(lr_shaps_y2_per_fold[i]).to_excel(writer, sheet_name=sheet_name, index=False)

with pd.ExcelWriter(f'results/{course_name}_SHAP_scores_LR_{year_finish}_retrained_per_fold.xlsx', engine='xlsxwriter') as writer:
    for i in range(10):

        sheet_name = f'Sheet{i+1}'  
        pd.DataFrame(lr_shaps_y2_retrained_per_fold[i]).to_excel(writer, sheet_name=sheet_name, index=False)

with pd.ExcelWriter(f'results/{course_name}_SHAP_scores_LR_{year_finish}_cv_per_fold.xlsx', engine='xlsxwriter') as writer:
    for i in range(10):

        sheet_name = f'Sheet{i+1}' 
        pd.DataFrame(lr_shaps_y2_cv_per_fold[i]).to_excel(writer, sheet_name=sheet_name, index=False)

In [66]:
lr_psi = pd.DataFrame({'Model X on data X vs. Model X on data X+1':lr_psi_scores_y1_y2, 
                        'Model X on data X vs. Model X retrained on data X+1':lr_psi_scores_y1_y2_retrained,
                          'Model X on data X vs. Model X+1 on data X+1':lr_psi_scores_y1_y2_cv})
lr_psi.to_excel(f'results/{course_name} {year_start} vs. {year_finish} LR PSI.xlsx')

In [67]:
with open(f'results/{course_name} {year_start} LR train indices.pickle', 'wb') as handle:
    pickle.dump(lr_ix_training_y1, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(f'results/{course_name} {year_start} LR test indices.pickle', 'wb') as handle:
    pickle.dump(lr_ix_test_y1, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(f'results/{course_name} {year_finish} LR train indices.pickle', 'wb') as handle:
    pickle.dump(lr_ix_training_y2, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(f'results/{course_name} {year_finish} LR test indices.pickle', 'wb') as handle:
    pickle.dump(lr_ix_test_y2, handle, protocol=pickle.HIGHEST_PROTOCOL)    

### SVM

In [ ]:
svm_outer_scores_y1, svm_shap_scores_y1, svm_outer_scores_y2, svm_shap_scores_y2, svm_outer_scores_y2_retrained, \
      svm_shap_scores_y2_retrained, svm_outer_scores_y2_cv, svm_shap_scores_y2_cv, svm_ix_training_y1, svm_ix_test_y1, \
          svm_ix_training_y2, svm_ix_test_y2, svm_shaps_y1_per_fold, svm_shaps_y2_per_fold,\
          svm_shaps_y2_retrained_per_fold, svm_shaps_y2_cv_per_fold,\
            svm_psi_scores_y1_y2,\
              svm_psi_scores_y1_y2_retrained, svm_psi_scores_y1_y2_cv = run_models_scaled(svm.SVC(probability=True), 'SVM', [{'C': [0.1, 1, 10, 100], 
                                                                                                    'gamma': [1, 0.1, 0.01, 0.001],
                                                                                                    'kernel': ['rbf', 'poly', 'sigmoid']}])

In [69]:
feature_names = independent_features

resultX = pd.DataFrame(svm_shap_scores_y1, columns = feature_names)
vals = np.abs(resultX.values).mean(0)
shap_importance_svm = pd.DataFrame(list(zip(feature_names, vals)),
                                  columns=['col_name','feature_importance_vals'])
shap_importance_svm.sort_values(by=['feature_importance_vals'],
                               ascending=False, inplace=True)
shap_importance_svm['Rank'] = len(stats.rankdata(shap_importance_svm['feature_importance_vals'])) \
      - stats.rankdata(shap_importance_svm['feature_importance_vals']) + 1

resultX = pd.DataFrame(svm_shap_scores_y2, columns = feature_names)
vals = np.abs(resultX.values).mean(0)
shap_importance_svm_y2 = pd.DataFrame(list(zip(feature_names, vals)),
                                  columns=['col_name','feature_importance_vals'])
shap_importance_svm_y2.sort_values(by=['feature_importance_vals'],
                               ascending=False, inplace=True)
shap_importance_svm_y2['Rank'] = len(stats.rankdata(shap_importance_svm_y2['feature_importance_vals'])) \
    - stats.rankdata(shap_importance_svm_y2['feature_importance_vals']) + 1

resultX = pd.DataFrame(svm_shap_scores_y2_retrained, columns = feature_names)
vals = np.abs(resultX.values).mean(0)
shap_importance_retrained_svm_y2 = pd.DataFrame(list(zip(feature_names, vals)),
                                  columns=['col_name','feature_importance_vals'])
shap_importance_retrained_svm_y2.sort_values(by=['feature_importance_vals'],
                               ascending=False, inplace=True)
shap_importance_retrained_svm_y2['Rank'] = len(stats.rankdata(shap_importance_retrained_svm_y2['feature_importance_vals'])) \
    - stats.rankdata(shap_importance_retrained_svm_y2['feature_importance_vals']) + 1

resultX = pd.DataFrame(svm_shap_scores_y2_cv, columns = feature_names)
vals = np.abs(resultX.values).mean(0)
shap_importance_svm_cv_y2 = pd.DataFrame(list(zip(feature_names, vals)),
                                  columns=['col_name','feature_importance_vals'])
shap_importance_svm_cv_y2.sort_values(by=['feature_importance_vals'],
                               ascending=False, inplace=True)
shap_importance_svm_cv_y2['Rank'] = len(stats.rankdata(shap_importance_svm_cv_y2['feature_importance_vals'])) \
      - stats.rankdata(shap_importance_svm_cv_y2['feature_importance_vals']) + 1

In [70]:
pd.DataFrame(svm_shap_scores_y1, columns=independent_features).to_excel(f'results/{course_name}_SHAP_scores_SVM_{year_start}.xlsx')
pd.DataFrame(svm_shap_scores_y2, columns=independent_features).to_excel(f'results/{course_name}_SHAP_scores_SVM_{year_finish}.xlsx')
pd.DataFrame(svm_shap_scores_y2_retrained, columns=independent_features).to_excel(f'results/{course_name}_SHAP_scores_SVM_{year_finish}_retrained.xlsx')
pd.DataFrame(svm_shap_scores_y2_cv, columns=independent_features).to_excel(f'results/{course_name}_SHAP_scores_SVM_{year_finish}_cv.xlsx')
pd.DataFrame({f'SVM {year_start}': svm_outer_scores_y1, f'SVM {year_finish}': svm_outer_scores_y2, 
              f'SVM {year_finish} retrained': svm_outer_scores_y2_retrained,
                f'SVM {year_finish} cv': svm_outer_scores_y2_cv }).to_excel(f'results/{course_name}_SVM_performance.xlsx')

In [71]:
with pd.ExcelWriter(f'results/{course_name}_SHAP_scores_SVM_{year_start}_per_fold.xlsx', engine='xlsxwriter') as writer:
    for i in range(10):

        sheet_name = f'Sheet{i+1}' 
        pd.DataFrame(svm_shaps_y1_per_fold[i]).to_excel(writer, sheet_name=sheet_name, index=False)

with pd.ExcelWriter(f'results/{course_name}_SHAP_scores_SVM_{year_finish}_per_fold.xlsx', engine='xlsxwriter') as writer:
    for i in range(10):

        sheet_name = f'Sheet{i+1}'  
        pd.DataFrame(svm_shaps_y2_per_fold[i]).to_excel(writer, sheet_name=sheet_name, index=False)

with pd.ExcelWriter(f'results/{course_name}_SHAP_scores_SVM_{year_finish}_retrained_per_fold.xlsx', engine='xlsxwriter') as writer:
    for i in range(10):

        sheet_name = f'Sheet{i+1}'  
        pd.DataFrame(svm_shaps_y2_retrained_per_fold[i]).to_excel(writer, sheet_name=sheet_name, index=False)

with pd.ExcelWriter(f'results/{course_name}_SHAP_scores_SVM_{year_finish}_cv_per_fold.xlsx', engine='xlsxwriter') as writer:
    for i in range(10):

        sheet_name = f'Sheet{i+1}' 
        pd.DataFrame(svm_shaps_y2_cv_per_fold[i]).to_excel(writer, sheet_name=sheet_name, index=False)

In [72]:
svm_psi = pd.DataFrame({'Model X on data X vs. Model X on data X+1':svm_psi_scores_y1_y2, 
                        'Model X on data X vs. Model X retrained on data X+1':svm_psi_scores_y1_y2_retrained,
                          'Model X on data X vs. Model X+1 on data X+1':svm_psi_scores_y1_y2_cv})
svm_psi.to_excel(f'results/{course_name} {year_start} vs. {year_finish} SVM PSI.xlsx')

In [73]:
with open(f'results/{course_name} {year_start} SVM train indices.pickle', 'wb') as handle:
    pickle.dump(svm_ix_training_y1, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(f'results/{course_name} {year_start} SVM test indices.pickle', 'wb') as handle:
    pickle.dump(svm_ix_test_y1, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(f'results/{course_name} {year_finish} SVM train indices.pickle', 'wb') as handle:
    pickle.dump(svm_ix_training_y2, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(f'results/{course_name} {year_finish} SVM test indices.pickle', 'wb') as handle:
    pickle.dump(svm_ix_test_y2, handle, protocol=pickle.HIGHEST_PROTOCOL)    

### KNN

In [ ]:
knn_outer_scores_y1, knn_shap_scores_y1, knn_outer_scores_y2, knn_shap_scores_y2, knn_outer_scores_y2_retrained,\
      knn_shap_scores_y2_retrained, knn_outer_scores_y2_cv, knn_shap_scores_y2_cv, knn_ix_training_y1, knn_ix_test_y1,\
          knn_ix_training_y2, knn_ix_test_y2, knn_shaps_y1_per_fold, knn_shaps_y2_per_fold,\
          knn_shaps_y2_retrained_per_fold, knn_shaps_y2_cv_per_fold,\
            knn_psi_scores_y1_y2,\
              knn_psi_scores_y1_y2_retrained, knn_psi_scores_y1_y2_cv = run_models_scaled(neighbors.KNeighborsClassifier(), 'KNN', [{'n_neighbors': range(1,30, 5), 
                                                                                                        'weights': ['uniform', 'distance']}])

In [75]:
feature_names = independent_features

resultX = pd.DataFrame(knn_shap_scores_y1, columns = feature_names)
vals = np.abs(resultX.values).mean(0)
shap_importance_knn = pd.DataFrame(list(zip(feature_names, vals)),
                                  columns=['col_name','feature_importance_vals'])
shap_importance_knn.sort_values(by=['feature_importance_vals'],
                               ascending=False, inplace=True)
shap_importance_knn['Rank'] = len(stats.rankdata(shap_importance_knn['feature_importance_vals'])) \
    - stats.rankdata(shap_importance_knn['feature_importance_vals']) + 1

resultX = pd.DataFrame(knn_shap_scores_y2, columns = feature_names)
vals = np.abs(resultX.values).mean(0)
shap_importance_knn_y2 = pd.DataFrame(list(zip(feature_names, vals)),
                                  columns=['col_name','feature_importance_vals'])
shap_importance_knn_y2.sort_values(by=['feature_importance_vals'],
                               ascending=False, inplace=True)
shap_importance_knn_y2['Rank'] = len(stats.rankdata(shap_importance_knn_y2['feature_importance_vals'])) \
      - stats.rankdata(shap_importance_knn_y2['feature_importance_vals']) + 1

resultX = pd.DataFrame(knn_shap_scores_y2_retrained, columns = feature_names)
vals = np.abs(resultX.values).mean(0)
shap_importance_retrained_knn_y2 = pd.DataFrame(list(zip(feature_names, vals)),
                                  columns=['col_name','feature_importance_vals'])
shap_importance_retrained_knn_y2.sort_values(by=['feature_importance_vals'],
                               ascending=False, inplace=True)
shap_importance_retrained_knn_y2['Rank'] = len(stats.rankdata(shap_importance_retrained_knn_y2['feature_importance_vals'])) \
      - stats.rankdata(shap_importance_retrained_knn_y2['feature_importance_vals']) + 1

resultX = pd.DataFrame(knn_shap_scores_y2_cv, columns = feature_names)
vals = np.abs(resultX.values).mean(0)
shap_importance_knn_cv_y2 = pd.DataFrame(list(zip(feature_names, vals)),
                                  columns=['col_name','feature_importance_vals'])
shap_importance_knn_cv_y2.sort_values(by=['feature_importance_vals'],
                               ascending=False, inplace=True)
shap_importance_knn_cv_y2['Rank'] = len(stats.rankdata(shap_importance_knn_cv_y2['feature_importance_vals'])) \
    - stats.rankdata(shap_importance_knn_cv_y2['feature_importance_vals']) + 1

In [76]:
pd.DataFrame(knn_shap_scores_y1, columns=independent_features).to_excel(f'results/{course_name}_SHAP_scores_KNN_{year_start}.xlsx')
pd.DataFrame(knn_shap_scores_y2, columns=independent_features).to_excel(f'results/{course_name}_SHAP_scores_KNN_{year_finish}.xlsx')
pd.DataFrame(knn_shap_scores_y2_retrained, columns=independent_features).to_excel(f'results/{course_name}_SHAP_scores_KNN_{year_finish}_retrained.xlsx')
pd.DataFrame(knn_shap_scores_y2_cv, columns=independent_features).to_excel(f'results/{course_name}_SHAP_scores_KNN_{year_finish}_cv.xlsx')
pd.DataFrame({f'KNN {year_start}': knn_outer_scores_y1, f'KNN {year_finish}': knn_outer_scores_y2, 
              f'KNN {year_finish} retrained': knn_outer_scores_y2_retrained,
                f'KNN {year_finish} cv': knn_outer_scores_y2_cv }).to_excel(f'results/{course_name}_KNN_performance.xlsx')

In [77]:
with pd.ExcelWriter(f'results/{course_name}_SHAP_scores_KNN_{year_start}_per_fold.xlsx', engine='xlsxwriter') as writer:
    for i in range(10):

        sheet_name = f'Sheet{i+1}' 
        pd.DataFrame(knn_shaps_y1_per_fold[i]).to_excel(writer, sheet_name=sheet_name, index=False)

with pd.ExcelWriter(f'results/{course_name}_SHAP_scores_KNN_{year_finish}_per_fold.xlsx', engine='xlsxwriter') as writer:
    for i in range(10):

        sheet_name = f'Sheet{i+1}'  
        pd.DataFrame(knn_shaps_y2_per_fold[i]).to_excel(writer, sheet_name=sheet_name, index=False)

with pd.ExcelWriter(f'results/{course_name}_SHAP_scores_KNN_{year_finish}_retrained_per_fold.xlsx', engine='xlsxwriter') as writer:
    for i in range(10):

        sheet_name = f'Sheet{i+1}'  
        pd.DataFrame(knn_shaps_y2_retrained_per_fold[i]).to_excel(writer, sheet_name=sheet_name, index=False)

with pd.ExcelWriter(f'results/{course_name}_SHAP_scores_KNN_{year_finish}_cv_per_fold.xlsx', engine='xlsxwriter') as writer:
    for i in range(10):

        sheet_name = f'Sheet{i+1}' 
        pd.DataFrame(knn_shaps_y2_cv_per_fold[i]).to_excel(writer, sheet_name=sheet_name, index=False)

In [78]:
knn_psi = pd.DataFrame({'Model X on data X vs. Model X on data X+1':knn_psi_scores_y1_y2, 
                        'Model X on data X vs. Model X retrained on data X+1':knn_psi_scores_y1_y2_retrained,
                          'Model X on data X vs. Model X+1 on data X+1':knn_psi_scores_y1_y2_cv})
knn_psi.to_excel(f'results/{course_name} {year_start} vs. {year_finish} KNN PSI.xlsx')

In [79]:
with open(f'results/{course_name} {year_start} KNN train indices.pickle', 'wb') as handle:
    pickle.dump(knn_ix_training_y1, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(f'results/{course_name} {year_start} KNN test indices.pickle', 'wb') as handle:
    pickle.dump(knn_ix_test_y1, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(f'results/{course_name} {year_finish} KNN train indices.pickle', 'wb') as handle:
    pickle.dump(knn_ix_training_y2, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(f'results/{course_name} {year_finish} KNN test indices.pickle', 'wb') as handle:
    pickle.dump(knn_ix_test_y2, handle, protocol=pickle.HIGHEST_PROTOCOL)    

### MLP

In [ ]:
mlp_outer_scores_y1, mlp_shap_scores_y1, mlp_outer_scores_y2, mlp_shap_scores_y2, mlp_outer_scores_y2_retrained,\
      mlp_shap_scores_y2_retrained, mlp_outer_scores_y2_cv, mlp_shap_scores_y2_cv, mlp_ix_training_y1, mlp_ix_test_y1,\
          mlp_ix_training_y2, mlp_ix_test_y2, mlp_shaps_y1_per_fold, mlp_shaps_y2_per_fold,\
          mlp_shaps_y2_retrained_per_fold, mlp_shaps_y2_cv_per_fold,\
            mlp_psi_scores_y1_y2,\
              mlp_psi_scores_y1_y2_retrained, mlp_psi_scores_y1_y2_cv = run_models_scaled(MLPClassifier(batch_size='auto', warm_start=True, max_iter=400), 'MLP', [{
    'hidden_layer_sizes': [(int(len(X_y1)/2),), (int(len(X_y1)/2), int(len(X_y1)/2/2),)],
    'activation': ['identity', 'logistic', 'tanh', 'relu'],
    'alpha': [0.000001, 0.00001, 0.0001],
    'solver': ['lbfgs', 'sgd', 'adam'],
    'learning_rate_init':[0.001, 0.0001]}])

In [81]:
feature_names = independent_features

resultX = pd.DataFrame(mlp_shap_scores_y1, columns = feature_names)
vals = np.abs(resultX.values).mean(0)
shap_importance_mlp = pd.DataFrame(list(zip(feature_names, vals)),
                                  columns=['col_name','feature_importance_vals'])
shap_importance_mlp.sort_values(by=['feature_importance_vals'],
                               ascending=False, inplace=True)
shap_importance_mlp['Rank'] = len(stats.rankdata(shap_importance_mlp['feature_importance_vals'])) \
    - stats.rankdata(shap_importance_mlp['feature_importance_vals']) + 1

resultX = pd.DataFrame(mlp_shap_scores_y2, columns = feature_names)
vals = np.abs(resultX.values).mean(0)
shap_importance_mlp_y2 = pd.DataFrame(list(zip(feature_names, vals)),
                                  columns=['col_name','feature_importance_vals'])
shap_importance_mlp_y2.sort_values(by=['feature_importance_vals'],
                               ascending=False, inplace=True)
shap_importance_mlp_y2['Rank'] = len(stats.rankdata(shap_importance_mlp_y2['feature_importance_vals'])) \
    - stats.rankdata(shap_importance_mlp_y2['feature_importance_vals']) + 1

resultX = pd.DataFrame(mlp_shap_scores_y2_retrained, columns = feature_names)
vals = np.abs(resultX.values).mean(0)
shap_importance_retrained_mlp_y2 = pd.DataFrame(list(zip(feature_names, vals)),
                                  columns=['col_name','feature_importance_vals'])
shap_importance_retrained_mlp_y2.sort_values(by=['feature_importance_vals'],
                               ascending=False, inplace=True)
shap_importance_retrained_mlp_y2['Rank'] = len(stats.rankdata(shap_importance_retrained_mlp_y2['feature_importance_vals'])) \
      - stats.rankdata(shap_importance_retrained_mlp_y2['feature_importance_vals']) + 1

resultX = pd.DataFrame(mlp_shap_scores_y2_cv, columns = feature_names)
vals = np.abs(resultX.values).mean(0)
shap_importance_mlp_cv_y2 = pd.DataFrame(list(zip(feature_names, vals)),
                                  columns=['col_name','feature_importance_vals'])
shap_importance_mlp_cv_y2.sort_values(by=['feature_importance_vals'],
                               ascending=False, inplace=True)
shap_importance_mlp_cv_y2['Rank'] = len(stats.rankdata(shap_importance_mlp_cv_y2['feature_importance_vals'])) \
    - stats.rankdata(shap_importance_mlp_cv_y2['feature_importance_vals']) + 1

In [82]:
pd.DataFrame(mlp_shap_scores_y1, columns=independent_features).to_excel(f'results/{course_name}_SHAP_scores_MLP_{year_start}.xlsx')
pd.DataFrame(mlp_shap_scores_y2, columns=independent_features).to_excel(f'results/{course_name}_SHAP_scores_MLP_{year_finish}.xlsx')
pd.DataFrame(mlp_shap_scores_y2_retrained, columns=independent_features).to_excel(f'results/{course_name}_SHAP_scores_MLP_{year_finish}_retrained.xlsx')
pd.DataFrame(mlp_shap_scores_y2_cv, columns=independent_features).to_excel(f'results/{course_name}_SHAP_scores_MLP_{year_finish}_cv.xlsx')
pd.DataFrame({f'MLP {year_start}': mlp_outer_scores_y1, f'MLP {year_finish}': mlp_outer_scores_y2, 
              f'MLP {year_finish} retrained': mlp_outer_scores_y2_retrained,
                f'MLP {year_finish} cv': mlp_outer_scores_y2_cv }).to_excel(f'results/{course_name}_MLP_performance.xlsx')

In [83]:
with pd.ExcelWriter(f'results/{course_name}_SHAP_scores_MLP_{year_start}_per_fold.xlsx', engine='xlsxwriter') as writer:
    for i in range(10):

        sheet_name = f'Sheet{i+1}' 
        pd.DataFrame(mlp_shaps_y1_per_fold[i]).to_excel(writer, sheet_name=sheet_name, index=False)

with pd.ExcelWriter(f'results/{course_name}_SHAP_scores_MLP_{year_finish}_per_fold.xlsx', engine='xlsxwriter') as writer:
    for i in range(10):

        sheet_name = f'Sheet{i+1}'  
        pd.DataFrame(mlp_shaps_y2_per_fold[i]).to_excel(writer, sheet_name=sheet_name, index=False)

with pd.ExcelWriter(f'results/{course_name}_SHAP_scores_MLP_{year_finish}_retrained_per_fold.xlsx', engine='xlsxwriter') as writer:
    for i in range(10):

        sheet_name = f'Sheet{i+1}'  
        pd.DataFrame(mlp_shaps_y2_retrained_per_fold[i]).to_excel(writer, sheet_name=sheet_name, index=False)

with pd.ExcelWriter(f'results/{course_name}_SHAP_scores_MLP_{year_finish}_cv_per_fold.xlsx', engine='xlsxwriter') as writer:
    for i in range(10):

        sheet_name = f'Sheet{i+1}' 
        pd.DataFrame(mlp_shaps_y2_cv_per_fold[i]).to_excel(writer, sheet_name=sheet_name, index=False)

In [84]:
mlp_psi = pd.DataFrame({'Model X on data X vs. Model X on data X+1':mlp_psi_scores_y1_y2, 
                        'Model X on data X vs. Model X retrained on data X+1':mlp_psi_scores_y1_y2_retrained,
                          'Model X on data X vs. Model X+1 on data X+1':mlp_psi_scores_y1_y2_cv})
mlp_psi.to_excel(f'results/{course_name} {year_start} vs. {year_finish} MLP PSI.xlsx')

In [85]:
with open(f'results/{course_name} {year_start} MLP train indices.pickle', 'wb') as handle:
    pickle.dump(mlp_ix_training_y1, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(f'results/{course_name} {year_start} MLP test indices.pickle', 'wb') as handle:
    pickle.dump(mlp_ix_test_y1, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(f'results/{course_name} {year_finish} MLP train indices.pickle', 'wb') as handle:
    pickle.dump(mlp_ix_training_y2, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(f'results/{course_name} {year_finish} MLP test indices.pickle', 'wb') as handle:
    pickle.dump(mlp_ix_test_y2, handle, protocol=pickle.HIGHEST_PROTOCOL)    

### XGBoost

In [ ]:
xgb_outer_scores_y1, xgb_shap_scores_y1, xgb_outer_scores_y2, xgb_shap_scores_y2, xgb_outer_scores_y2_retrained,\
      xgb_shap_scores_y2_retrained, xgb_outer_scores_y2_cv, xgb_shap_scores_y2_cv, xgb_ix_training_y1, xgb_ix_test_y1,\
          xgb_ix_training_y2, xgb_ix_test_y2, xgb_shaps_y1_per_fold, xgb_shaps_y2_per_fold,\
          xgb_shaps_y2_retrained_per_fold, xgb_shaps_y2_cv_per_fold,\
            xgb_psi_scores_y1_y2,\
              xgb_psi_scores_y1_y2_retrained, xgb_psi_scores_y1_y2_cv = run_models_scaled(GradientBoostingClassifier(), 'XGB', 
                                                                                        [{
                                                                                          'learning_rate':[0.1,0.01],
                                                                                          'n_estimators': [50, 100, 200],
                                                                                          'min_samples_split': [2, 5, 10],
                                                                                          'max_depth': [3, 5, 10, None], 
                                                                                          'max_features': ['sqrt','log2', None]}])

In [87]:
feature_names = independent_features

resultX = pd.DataFrame(xgb_shap_scores_y1, columns = feature_names)
vals = np.abs(resultX.values).mean(0)
shap_importance_xgb = pd.DataFrame(list(zip(feature_names, vals)),
                                  columns=['col_name','feature_importance_vals'])
shap_importance_xgb.sort_values(by=['feature_importance_vals'],
                               ascending=False, inplace=True)
shap_importance_xgb['Rank'] = len(stats.rankdata(shap_importance_xgb['feature_importance_vals'])) \
    - stats.rankdata(shap_importance_xgb['feature_importance_vals']) + 1

resultX = pd.DataFrame(xgb_shap_scores_y2, columns = feature_names)
vals = np.abs(resultX.values).mean(0)
shap_importance_xgb_y2 = pd.DataFrame(list(zip(feature_names, vals)),
                                  columns=['col_name','feature_importance_vals'])
shap_importance_xgb_y2.sort_values(by=['feature_importance_vals'],
                               ascending=False, inplace=True)
shap_importance_xgb_y2['Rank'] = len(stats.rankdata(shap_importance_xgb_y2['feature_importance_vals'])) \
    - stats.rankdata(shap_importance_xgb_y2['feature_importance_vals']) + 1

resultX = pd.DataFrame(xgb_shap_scores_y2_retrained, columns = feature_names)
vals = np.abs(resultX.values).mean(0)
shap_importance_retrained_xgb_y2 = pd.DataFrame(list(zip(feature_names, vals)),
                                  columns=['col_name','feature_importance_vals'])
shap_importance_retrained_xgb_y2.sort_values(by=['feature_importance_vals'],
                               ascending=False, inplace=True)
shap_importance_retrained_xgb_y2['Rank'] = len(stats.rankdata(shap_importance_retrained_xgb_y2['feature_importance_vals'])) \
      - stats.rankdata(shap_importance_retrained_xgb_y2['feature_importance_vals']) + 1

resultX = pd.DataFrame(xgb_shap_scores_y2_cv, columns = feature_names)
vals = np.abs(resultX.values).mean(0)
shap_importance_xgb_cv_y2 = pd.DataFrame(list(zip(feature_names, vals)),
                                  columns=['col_name','feature_importance_vals'])
shap_importance_xgb_cv_y2.sort_values(by=['feature_importance_vals'],
                               ascending=False, inplace=True)
shap_importance_xgb_cv_y2['Rank'] = len(stats.rankdata(shap_importance_xgb_cv_y2['feature_importance_vals'])) \
    - stats.rankdata(shap_importance_xgb_cv_y2['feature_importance_vals']) + 1

In [88]:
pd.DataFrame(xgb_shap_scores_y1, columns=independent_features).to_excel(f'results/{course_name}_SHAP_scores_XGB_{year_start}.xlsx')
pd.DataFrame(xgb_shap_scores_y2, columns=independent_features).to_excel(f'results/{course_name}_SHAP_scores_XGB_{year_finish}.xlsx')
pd.DataFrame(xgb_shap_scores_y2_retrained, columns=independent_features).to_excel(f'results/{course_name}_SHAP_scores_XGB_{year_finish}_retrained.xlsx')
pd.DataFrame(xgb_shap_scores_y2_cv, columns=independent_features).to_excel(f'results/{course_name}_SHAP_scores_XGB_{year_finish}_cv.xlsx')
pd.DataFrame({f'XGB {year_start}': xgb_outer_scores_y1, f'XGB {year_finish}': xgb_outer_scores_y2, 
              f'XGB {year_finish} retrained': xgb_outer_scores_y2_retrained,
                f'XGB {year_finish} cv': xgb_outer_scores_y2_cv }).to_excel(f'results/{course_name}_XGB_performance.xlsx')

In [89]:
with pd.ExcelWriter(f'results/{course_name}_SHAP_scores_XGB_{year_start}_per_fold.xlsx', engine='xlsxwriter') as writer:
    for i in range(10):

        sheet_name = f'Sheet{i+1}' 
        pd.DataFrame(xgb_shaps_y1_per_fold[i]).to_excel(writer, sheet_name=sheet_name, index=False)

with pd.ExcelWriter(f'results/{course_name}_SHAP_scores_XGB_{year_finish}_per_fold.xlsx', engine='xlsxwriter') as writer:
    for i in range(10):

        sheet_name = f'Sheet{i+1}'  
        pd.DataFrame(xgb_shaps_y2_per_fold[i]).to_excel(writer, sheet_name=sheet_name, index=False)

with pd.ExcelWriter(f'results/{course_name}_SHAP_scores_XGB_{year_finish}_retrained_per_fold.xlsx', engine='xlsxwriter') as writer:
    for i in range(10):

        sheet_name = f'Sheet{i+1}'  
        pd.DataFrame(xgb_shaps_y2_retrained_per_fold[i]).to_excel(writer, sheet_name=sheet_name, index=False)

with pd.ExcelWriter(f'results/{course_name}_SHAP_scores_XGB_{year_finish}_cv_per_fold.xlsx', engine='xlsxwriter') as writer:
    for i in range(10):

        sheet_name = f'Sheet{i+1}' 
        pd.DataFrame(xgb_shaps_y2_cv_per_fold[i]).to_excel(writer, sheet_name=sheet_name, index=False)

In [90]:
xgb_psi = pd.DataFrame({'Model X on data X vs. Model X on data X+1':xgb_psi_scores_y1_y2, 
                        'Model X on data X vs. Model X retrained on data X+1':xgb_psi_scores_y1_y2_retrained,
                          'Model X on data X vs. Model X+1 on data X+1':xgb_psi_scores_y1_y2_cv})
xgb_psi.to_excel(f'results/{course_name} {year_start} vs. {year_finish} XGB PSI.xlsx')

In [91]:
with open(f'results/{course_name} {year_start} XGB train indices.pickle', 'wb') as handle:
    pickle.dump(xgb_ix_training_y1, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(f'results/{course_name} {year_start} XGB test indices.pickle', 'wb') as handle:
    pickle.dump(xgb_ix_test_y1, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(f'results/{course_name} {year_finish} XGB train indices.pickle', 'wb') as handle:
    pickle.dump(xgb_ix_training_y2, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(f'results/{course_name} {year_finish} XGB test indices.pickle', 'wb') as handle:
    pickle.dump(xgb_ix_test_y2, handle, protocol=pickle.HIGHEST_PROTOCOL)    

### TabNet

In [50]:
def Objective(trial, X, y):
    n_da = trial.suggest_int("n_da", 8, 32, step=8)
    n_steps = trial.suggest_int("n_steps", 1, 3, step=1)
    gamma = trial.suggest_float("gamma", 1., 1.4, step=0.2)
    n_shared = trial.suggest_int("n_shared", 1, 3)
    lambda_sparse = trial.suggest_float("lambda_sparse", 1e-6, 1e-3, log=True)
    
    tabnet_params = dict(n_d=n_da, n_a=n_da, n_steps=n_steps, gamma=gamma,
                     lambda_sparse=lambda_sparse,
                     optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
                     n_shared=n_shared,
                     scheduler_params=dict(mode="min",
                                           patience=trial.suggest_int("patienceScheduler",low=10,high=20),
                                           min_lr=1e-5,
                                           factor=0.5,),
                     verbose=0,
                     ) 
    
    kf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True) 
    CV_score_array    =[]
    for train_index, test_index in kf.split(X, y):
        X_train, X_valid = X[train_index], X[test_index]
        y_train, y_valid = y[train_index], y[test_index]
        regressor = TabNetClassifier(**tabnet_params)
        regressor.fit(X_train=X_train, y_train=y_train.flatten(),
                  eval_set=[(X_valid, y_valid.flatten())],
                  patience=trial.suggest_int("patience",low=15,high=30), max_epochs=trial.suggest_int('epochs', 1, 100),
                  eval_metric=['balanced_accuracy'],
                  batch_size=64)
        CV_score_array.append(regressor.best_cost)
    avg = np.mean(CV_score_array)
    return avg

In [51]:
def run_models_TabNet(model, model_name):
    
    outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
    psi_scores_y1_y2 = []
    psi_scores_y1_y2_retrained = []
    psi_scores_y1_y2_cv = []

    outer_scores_y1 = []
    shap_scores_y1 = []

    outer_scores_y2 = []
    shap_scores_y2 = []

    outer_scores_y2_retrained = []
    shap_scores_y2_retrained = []

    outer_scores_y2_cv = []
    shap_scores_y2_cv = []

    ix_training_y1, ix_test_y1 = [], []
    ix_training_y2, ix_test_y2 = [], []

    shaps_y1_per_fold = {}
    shaps_y2_per_fold = {}
    shaps_y2_retrained_per_fold = {}
    shaps_y2_cv_per_fold = {}

    for fold in outer_cv.split(X_y1, y_y1):
        ix_training_y1.append(fold[0]), ix_test_y1.append(fold[1])

    for fold in outer_cv.split(X_y2, y_y2):
        ix_training_y2.append(fold[0]), ix_test_y2.append(fold[1])

    
    for i in range(10):  

        train_idX_y1, test_idX_y1 = ix_training_y1[i], ix_test_y1[i]
        train_idX_y2, test_idX_y2 = ix_training_y2[i], ix_test_y2[i]
        
        # step 1)
        study = optuna.create_study(direction="maximize", study_name='TabNet optimization')
        study.optimize(lambda trial: Objective(trial,X_y1[train_idX_y1], y_y1[train_idX_y1].flatten()), timeout=6*60) 
        TabNet_params = study.best_params

        final_params = dict(n_d=TabNet_params['n_da'], n_a=TabNet_params['n_da'],
                     n_steps=TabNet_params['n_steps'], 
                     gamma=TabNet_params['gamma'],
                      n_shared=TabNet_params['n_shared'])
        epochs = TabNet_params['epochs']

        print(f'Step {i} for model {model_name}')
        print('\n        Best ACCURACY y1 model %.2f%%' % (study.best_value * 100))
        print('        Best parameters y1 model:', study.best_params)

        best_model = TabNetClassifier(**final_params)
        best_model.fit(X_train=X_y1[train_idX_y1], y_train=y_y1[train_idX_y1].flatten(), 
                    patience=TabNet_params['patience'], max_epochs=epochs,
                        batch_size=64,
                    eval_metric=['balanced_accuracy'])
        
        # step 2) 
        y_pred = best_model.predict(X_y1[test_idX_y1])
        outer_scores_y1.append(balanced_accuracy_score(y_y1[test_idX_y1], y_pred))
        print('AUC y1 (on outer test fold) %.2f%%' % (outer_scores_y1[-1]*100))

        # step 3) 
        explainer_y1 = shap.KernelExplainer(best_model.predict_proba, shap.sample(X_y1[train_idX_y1], 100))
        shap_values_y1 = explainer_y1.shap_values(X_y1[test_idX_y1])
        for SHAPs in shap_values_y1[1]:
            shap_scores_y1.append(SHAPs)
        shaps_y1_per_fold[i] = shap_values_y1[1]

        # step 4)  
        y_pred = best_model.predict(X_y2[test_idX_y2])
        outer_scores_y2.append(balanced_accuracy_score(y_y2[test_idX_y2], y_pred))
        print('ACCURACY y2 (on outer test fold) %.2f%%' % (outer_scores_y2[-1]*100))
        probas_y1 = best_model.predict_proba(X_y1[test_idX_y1])[:,1]
        probas_y2 = best_model.predict_proba(X_y2[test_idX_y2])[:,1]
        psi_scores_y1_y2.append(psi(probas_y1,
                                     probas_y2))
        
        # step 5)
        shap_values_y2 = explainer_y1.shap_values(X_y2[test_idX_y2])
        for SHAPs in shap_values_y2[1]:
            shap_scores_y2.append(SHAPs) 
        shaps_y2_per_fold[i] = shap_values_y2[1]

        # step 6)
        best_model.fit(X_y2[train_idX_y2], y_y2[train_idX_y2].flatten(),
                       patience=TabNet_params['patience'], max_epochs=epochs,
                        batch_size=64,
                    eval_metric=['balanced_accuracy'])

        # step 7)
        y_pred = best_model.predict(X_y2[test_idX_y2])
        outer_scores_y2_retrained.append(balanced_accuracy_score(y_y2[test_idX_y2], y_pred))
        print('ACCURACY y2 retrained (on outer test fold) %.2f%%' % (outer_scores_y2_retrained[-1]*100))
        probas_y2_retrained = best_model.predict_proba(X_y2[test_idX_y2])[:,1]
        psi_scores_y1_y2_retrained.append(psi(probas_y1,
                                               probas_y2_retrained))

        # step 8)
        explainer_y2_retrained = shap.KernelExplainer(best_model.predict_proba, shap.sample(X_y2[train_idX_y2], 100))
        shap_values_y2_retrained = explainer_y2_retrained.shap_values(X_y2[test_idX_y2])
        for SHAPs in shap_values_y2_retrained[1]:
            shap_scores_y2_retrained.append(SHAPs) 
        shaps_y2_retrained_per_fold[i] = shap_values_y2_retrained[1]

        #step 9) 

        study = optuna.create_study(direction="maximize", study_name='TabNet optimization')
        study.optimize(lambda trial: Objective(trial,X_y2[train_idX_y2], y_y2[train_idX_y2].flatten()), timeout=6*60) 
        TabNet_params = study.best_params

        final_params = dict(n_d=TabNet_params['n_da'], n_a=TabNet_params['n_da'],
                     n_steps=TabNet_params['n_steps'], 
                     gamma=TabNet_params['gamma'],
                      n_shared=TabNet_params['n_shared'])
        epochs = TabNet_params['epochs']

        print('\n        Best ACCURACY y2 CV model %.2f%%' % (study.best_value * 100))
        print('        Best parameters:', study.best_params)

        best_model_y2 = TabNetClassifier(**final_params)
        best_model_y2.fit(X_train=X_y2[train_idX_y2], y_train=y_y2[train_idX_y2].flatten(), 
                    patience=TabNet_params['patience'], max_epochs=epochs,
                        batch_size=64,
                    eval_metric=['balanced_accuracy'])
        
        # step 10)
        y_pred = best_model_y2.predict(X_y2[test_idX_y2])
        outer_scores_y2_cv.append(balanced_accuracy_score(y_y2[test_idX_y2], y_pred))
        print('ACCURACY y2 after CV (on outer test fold) %.2f%%' % (outer_scores_y2_cv[-1]*100))
        probas_y2_cv = best_model_y2.predict_proba(X_y2[test_idX_y2])[:,1]
        psi_scores_y1_y2_cv.append(psi(probas_y1,
                                        probas_y2_cv))

        #step 11)
        explainer_y2_cv = shap.KernelExplainer(best_model_y2.predict_proba, shap.sample(X_y2[train_idX_y2], 100))
        shap_values_y2_cv = explainer_y2_cv.shap_values(X_y2[test_idX_y2])
        for SHAPs in shap_values_y2_cv[1]:
            shap_scores_y2_cv.append(SHAPs)
        shaps_y2_cv_per_fold[i] = shap_values_y2_cv[1]

    return outer_scores_y1, shap_scores_y1, outer_scores_y2,\
          shap_scores_y2, outer_scores_y2_retrained, shap_scores_y2_retrained,\
              outer_scores_y2_cv, shap_scores_y2_cv, ix_training_y1, ix_test_y1,\
                  ix_training_y2, ix_test_y2,shaps_y1_per_fold, shaps_y2_per_fold,\
          shaps_y2_retrained_per_fold, shaps_y2_cv_per_fold,\
          psi_scores_y1_y2,\
              psi_scores_y1_y2_retrained, psi_scores_y1_y2_cv

In [ ]:
tab_outer_scores_y1, tab_shap_scores_y1, tab_outer_scores_y2, tab_shap_scores_y2, tab_outer_scores_y2_retrained,\
      tab_shap_scores_y2_retrained, tab_outer_scores_y2_cv, tab_shap_scores_y2_cv, tab_ix_training_y1, tab_ix_test_y1,\
          tab_ix_training_y2, tab_ix_test_y2, tab_shaps_y1_per_fold, tab_shaps_y2_per_fold,\
          tab_shaps_y2_retrained_per_fold, tab_shaps_y2_cv_per_fold,\
            tab_psi_scores_y1_y2,\
              tab_psi_scores_y1_y2_retrained, tab_psi_scores_y1_y2_cv = run_models_TabNet( TabNetClassifier(verbose=0,seed=42), 'TabNet')

In [53]:
feature_names = independent_features

resultX = pd.DataFrame(tab_shap_scores_y1, columns = feature_names)
vals = np.abs(resultX.values).mean(0)
shap_importance_tab = pd.DataFrame(list(zip(feature_names, vals)),
                                  columns=['col_name','feature_importance_vals'])
shap_importance_tab.sort_values(by=['feature_importance_vals'],
                               ascending=False, inplace=True)
shap_importance_tab['Rank'] = len(stats.rankdata(shap_importance_tab['feature_importance_vals'])) \
    - stats.rankdata(shap_importance_tab['feature_importance_vals']) + 1

resultX = pd.DataFrame(tab_shap_scores_y2, columns = feature_names)
vals = np.abs(resultX.values).mean(0)
shap_importance_tab_y2 = pd.DataFrame(list(zip(feature_names, vals)),
                                  columns=['col_name','feature_importance_vals'])
shap_importance_tab_y2.sort_values(by=['feature_importance_vals'],
                               ascending=False, inplace=True)
shap_importance_tab_y2['Rank'] = len(stats.rankdata(shap_importance_tab_y2['feature_importance_vals'])) \
    - stats.rankdata(shap_importance_tab_y2['feature_importance_vals']) + 1

resultX = pd.DataFrame(tab_shap_scores_y2_retrained, columns = feature_names)
vals = np.abs(resultX.values).mean(0)
shap_importance_retrained_tab_y2 = pd.DataFrame(list(zip(feature_names, vals)),
                                  columns=['col_name','feature_importance_vals'])
shap_importance_retrained_tab_y2.sort_values(by=['feature_importance_vals'],
                               ascending=False, inplace=True)
shap_importance_retrained_tab_y2['Rank'] = len(stats.rankdata(shap_importance_retrained_tab_y2['feature_importance_vals'])) \
      - stats.rankdata(shap_importance_retrained_tab_y2['feature_importance_vals']) + 1

resultX = pd.DataFrame(tab_shap_scores_y2_cv, columns = feature_names)
vals = np.abs(resultX.values).mean(0)
shap_importance_tab_cv_y2 = pd.DataFrame(list(zip(feature_names, vals)),
                                  columns=['col_name','feature_importance_vals'])
shap_importance_tab_cv_y2.sort_values(by=['feature_importance_vals'],
                               ascending=False, inplace=True)
shap_importance_tab_cv_y2['Rank'] = len(stats.rankdata(shap_importance_tab_cv_y2['feature_importance_vals'])) \
    - stats.rankdata(shap_importance_tab_cv_y2['feature_importance_vals']) + 1

In [54]:
pd.DataFrame(tab_shap_scores_y1, columns=independent_features).to_excel(f'results/{course_name}_SHAP_scores_TAB_{year_start}.xlsx')
pd.DataFrame(tab_shap_scores_y2, columns=independent_features).to_excel(f'results/{course_name}_SHAP_scores_TAB_{year_finish}.xlsx')
pd.DataFrame(tab_shap_scores_y2_retrained, columns=independent_features).to_excel(f'results/{course_name}_SHAP_scores_TAB_{year_finish}_retrained.xlsx')
pd.DataFrame(tab_shap_scores_y2_cv, columns=independent_features).to_excel(f'results/{course_name}_SHAP_scores_TAB_{year_finish}_cv.xlsx')
pd.DataFrame({f'TAB {year_start}': tab_outer_scores_y1, f'TAB {year_finish}': tab_outer_scores_y2, 
              f'TAB {year_finish} retrained': tab_outer_scores_y2_retrained,
                f'TAB {year_finish} cv': tab_outer_scores_y2_cv }).to_excel(f'results/{course_name}_TAB_performance.xlsx')

In [55]:
with pd.ExcelWriter(f'results/{course_name}_SHAP_scores_TAB_{year_start}_per_fold.xlsx', engine='xlsxwriter') as writer:
    for i in range(10):

        sheet_name = f'Sheet{i+1}' 
        pd.DataFrame(tab_shaps_y1_per_fold[i]).to_excel(writer, sheet_name=sheet_name, index=False)

with pd.ExcelWriter(f'results/{course_name}_SHAP_scores_TAB_{year_finish}_per_fold.xlsx', engine='xlsxwriter') as writer:
    for i in range(10):

        sheet_name = f'Sheet{i+1}'  
        pd.DataFrame(tab_shaps_y2_per_fold[i]).to_excel(writer, sheet_name=sheet_name, index=False)

with pd.ExcelWriter(f'results/{course_name}_SHAP_scores_TAB_{year_finish}_retrained_per_fold.xlsx', engine='xlsxwriter') as writer:
    for i in range(10):

        sheet_name = f'Sheet{i+1}'  
        pd.DataFrame(tab_shaps_y2_retrained_per_fold[i]).to_excel(writer, sheet_name=sheet_name, index=False)

with pd.ExcelWriter(f'results/{course_name}_SHAP_scores_TAB_{year_finish}_cv_per_fold.xlsx', engine='xlsxwriter') as writer:
    for i in range(10):

        sheet_name = f'Sheet{i+1}' 
        pd.DataFrame(tab_shaps_y2_cv_per_fold[i]).to_excel(writer, sheet_name=sheet_name, index=False)

In [56]:
tab_psi = pd.DataFrame({'Model X on data X vs. Model X on data X+1':tab_psi_scores_y1_y2, 
                        'Model X on data X vs. Model X retrained on data X+1':tab_psi_scores_y1_y2_retrained,
                          'Model X on data X vs. Model X+1 on data X+1':tab_psi_scores_y1_y2_cv})
tab_psi.to_excel(f'results/{course_name} {year_start} vs. {year_finish} TAB PSI.xlsx')

In [57]:
with open(f'results/{course_name} {year_start} TAB train indices.pickle', 'wb') as handle:
    pickle.dump(tab_ix_training_y1, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(f'results/{course_name} {year_start} TAB test indices.pickle', 'wb') as handle:
    pickle.dump(tab_ix_test_y1, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(f'results/{course_name} {year_finish} TAB train indices.pickle', 'wb') as handle:
    pickle.dump(tab_ix_training_y2, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(f'results/{course_name} {year_finish} TAB test indices.pickle', 'wb') as handle:
    pickle.dump(tab_ix_test_y2, handle, protocol=pickle.HIGHEST_PROTOCOL)    